# XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.datasets import fetch_openml  # データセットのダウンロード取得用
import pandas as pd  # データフレーム操作用
import matplotlib.pyplot as plt  # グラフ描画用

In [ ]:
# 1. データセットの取得
dataset = fetch_openml("pima-indians-diabetes", version=1, as_frame=True)  # Pimaデータセットを取得
# こんな感じのデータです．8変数の説明変数．Outcomeは0 or 1
dataset.frame.head()  # データフレームの最初の5行を表示

In [ ]:
# 1.1 参考までに相関行列を表示
corr = dataset.frame.corr()
# 相関行列をヒートマップで表示．有効数字2桁
corr.style.format(precision=2).background_gradient(cmap="coolwarm")

In [ ]:
# 2. 説明変数をX，目的変数をyに分ける
X = dataset.data
y = dataset.target.astype("int")  # 目的変数は整数型に変換

# こっちの方が汎用性が高いです．PandasのDataFrameから変換する方法
df = dataset.frame
X = df.drop(columns=["Outcome"])  # Outcome列を除外
y = df["Outcome"].astype(int)  # Outcome列を抽出 & 整数型に変換

In [ ]:
# 3. 訓練データとテストデータに分割
from sklearn.model_selection import train_test_split  # データ分割用
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=500)  # 80%訓練，20%テスト

In [ ]:
# 4. xgboostを使って学習
model = XGBClassifier(
    n_estimators=100,  # 木の数
    max_depth=3,  # 木の深さ
    learning_rate=0.1,  # 学習率
    random_state=100,  # 再現性のための乱数シード
)
model.fit(X_train, y_train)  # 訓練データで学習


In [ ]:
# 5. テストデータで予測
y_pred = model.predict(X_test)  # 予測
# 6. 精度を評価
from sklearn.metrics import accuracy_score  # 精度評価用
accuracy = accuracy_score(y_test, y_pred)  # 精度計算
print(f"Accuracy: {accuracy:.3f}")  # 精度を表示(小数点以下3桁)

In [ ]:
# 7. 混同行列を表示
from sklearn.metrics import confusion_matrix  # 混同行列用
cm = confusion_matrix(y_test, y_pred)  # 混同行列を計算
plt.matshow(cm, cmap="Blues")  # 混同行列をヒートマップで表示
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(x=j, y=i, s=cm[i, j])

In [ ]:
# 8. 特徴量の重要度を表示
importances = model.feature_importances_  # 特徴量の重要度を取得
# 特徴量の重要度をデータフレームに変換
importance_df = pd.DataFrame(
    {
        "feature": X.columns,  # 特徴量名
        "importance": importances,  # 特徴量の重要度
    }
)
importance_df = importance_df.sort_values(by="importance", ascending=False)  # 重要度でソート
# 特徴量の重要度を棒グラフで表示
importance_df.plot(
    kind="barh",  # 横棒グラフ
    x="feature",  # x軸は特徴量名
    y="importance",  # y軸は重要度
    title="Feature Importance",  # タイトル
    figsize=(10, 6),  # サイズ
)
plt.xlabel("Importance")  # x軸ラベル
plt.ylabel("Feature")  # y軸ラベル

In [ ]:
# おまけ

# ipywidgetで dataset.feature_namesの8変数をスライダーで指定して，その変数の値を使って予測する
# 9. ipywidgetでスライダーを作成
from ipywidgets import interact, FloatSlider  # スライダー作成用
import numpy as np  # 数値計算用

# スライダーの範囲を指定
def create_slider(name, min_value, max_value):
    return FloatSlider(value=(min_value + max_value) / 2, min=min_value, max=max_value, step=0.1, description=name)

# スライダーの値を使って予測する関数
def predict_with_sliders(**kwargs):
    # スライダーの値を使って予測
    input_data = pd.DataFrame({name: [kwargs[name]] for name in dataset.feature_names})
    prediction = model.predict_proba(input_data)  # 確率(ぽい何か)を予測
    print(f"Prediction: {prediction[0][1]:.2f}")  # 予測結果を表示

# スライダーを作成
sliders = {name: create_slider(name, X[name].min(), X[name].max()) for name in dataset.feature_names}  # スライダーを作成

# スライダーを表示
interact(predict_with_sliders, **sliders)  # スライダーを表示
